In [ ]:
# imports

import pandas as pd
import numpy as np
import requests, json, os, sys, io, itertools, time, cx_Oracle, sqlalchemy
from bs4 import BeautifulSoup

In [2]:
# settings

requests.packages.urllib3.disable_warnings()
RBRISK_USER = os.getenv('RBRISK_USER')
RBRISK_PASS = os.getenv('RBRISK_PASS')
RBRISK_CON = cx_Oracle.connect(RBRISK_USER,
                                RBRISK_PASS,
                                'RBRISK',
                                encoding = 'utf8', 
                                nencoding = 'utf8')
ENGINE_RBRISK = sqlalchemy.create_engine('oracle+cx_oracle://{}:{}@RBRISK'.format(RBRISK_USER, RBRISK_PASS),
                                encoding='utf8', convert_unicode=True)
georgian_alphabet = 'აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ'
bog_translation = str.maketrans(''.join(chr(i) for i in range(ord('А'),ord('А')+33)),georgian_alphabet)
DISK_PREF = 'E:/Workspace/{}/'.format(os.getcwd().split('\')[-1])

In [1]:
# functions
def send_message(text, chat_id = 841752112, token = '1364696703:AAFQzOSQUPGKVLNnZ4UvD4eXYvS6YjxwygY'):
    import requests
    url = 'https://api.telegram.org/bot{}/sendMessage'.format(token)
    data = {'chat_id': chat_id, 'text': text}
    resp = requests.post(url, data, verify=False).json()
    return resp
def get_rates(date = pd.Timestamp('now').floor('d').isoformat()[:10]):
    q = '''
        select 
        t.rate_weight*t.ccy_rate rate, 
        t.ccy1  ccy
        from 
        krn_ccy_rates@xldr.bg t
        where t.effect_date = date '{}'
        '''.format(date)
    data = pd.read_sql(q, RBRISK_CON).set_index('CCY').RATE.to_dict()
    data['GEL'] = 1
    return data
